In [1]:
!pip install torch transformers pandas scikit-learn matplotlib


In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import plotly.express as px

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cpu


In [4]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from google.colab import files
uploaded = files.upload()  # Use this to upload your CSV file

# Load the dataset (assuming it's called 'sales_data.csv')
df = pd.read_csv('sales_data.csv', parse_dates=['date'])
df.head()


In [ ]:

df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by='date')
df.fillna(method='ffill', inplace=True)
print(df.describe())
print(df.info())


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(df['date'], df['sales'])
plt.title('Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.show()


In [ ]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df['sales_scaled'] = scaler.fit_transform(df[['sales']])
df[['date', 'sales', 'sales_scaled']].head()

In [ ]:
train_size = int(len(df) * 0.8)
train_df = df[:train_size]
test_df = df[train_size:]
print(f'Training set size: {len(train_df)}')
print(f'Testing set size: {len(test_df)}')

In [ ]:
def create_sequences(data, target_column, seq_length=30):
    sequences = []
    targets = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        target = data[i + seq_length]
        sequences.append(seq)
        targets.append(target)
    return np.array(sequences), np.array(targets)
seq_length = 30
X_train, y_train = create_sequences(train_df['sales_scaled'].values, train_df['sales_scaled'].values, seq_length)
X_test, y_test = create_sequences(test_df['sales_scaled'].values, test_df['sales_scaled'].values, seq_length)


print(f'Training sequences shape: {X_train.shape}')


In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

def tokenize_sequences(sequences, tokenizer):
    tokenized_sequences = []
    for sequence in sequences:
        sequence_str = ' '.join(map(str, sequence))  # Convert the sequence to a string
        tokenized_seq = tokenizer(sequence_str, return_tensors="pt")['input_ids']
        tokenized_sequences.append(tokenized_seq)
    return tokenized_sequences


tokenized_train_sequences = tokenize_sequences(X_train, tokenizer)
tokenized_test_sequences = tokenize_sequences(X_test, tokenizer)

print(tokenized_train_sequences[0])


In [ ]:
train_gpt2(model, tokenized_train_sequences, y_train, num_epochs=1, learning_rate=1e-5)


In [ ]:
from torch.optim import Adam
from torch.nn import functional as F

# Define a function to fine-tune GPT-2 on the time series data
def train_gpt2(model, tokenized_sequences, targets, num_epochs=3, learning_rate=1e-4):
    model.train()  # Set model to training mode
    optimizer = Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        total_loss = 0
        for i, (input_seq, target_seq) in enumerate(zip(tokenized_sequences, targets)):
            input_seq = input_seq.squeeze().to(device)  # Ensure correct dimensions for input
            target_seq = torch.tensor([target_seq]).to(device)

            optimizer.zero_grad()
            outputs = model(input_seq, labels=input_seq)
            loss = F.mse_loss(outputs.logits[:, -1].squeeze(), target_seq.float())  # Use the final token in the sequence for prediction
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(tokenized_sequences)}')

# Train the model on the training data
train_gpt2(model, tokenized_train_sequences, y_train, num_epochs=1, learning_rate


In [ ]:
# Function to generate predictions from GPT-2
def generate_forecasts(model, tokenized_sequences):
    model.eval()  # Set model to evaluation mode
    forecasts = []
    with torch.no_grad():
        for input_seq in tokenized_sequences:
            input_seq = input_seq.squeeze().to(device)
            outputs = model.generate(input_seq)
            forecast = float(tokenizer.decode(outputs[0], skip_special_tokens=True).split()[-1])
            forecasts.append(forecast)
    return forecasts

# Generate forecasts on the test data
test_forecasts = generate_forecasts(model, tokenized_test_sequences)

# Inverse scale the forecasts to get the actual sales values
test_forecasts = scaler.inverse_transform(np.array(test_forecasts).reshape(-1, 1))

# Display the forecasts
print(test_forecasts)


In [ ]:

plt.figure(figsize=(10, 6))
plt.plot(test_df['date'][seq_length:], test_df['sales'][seq_length:], label='Actual Sales')
plt.plot(test_df['date'][seq_length:], test_forecasts, label='Predicted Sales')
plt.title('Actual vs Predicted Sales')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Calculate errors
mse = mean_squared_error(test_df['sales'][seq_length:], test_forecasts)
mae = mean_absolute_error(test_df['sales'][seq_length:], test_forecasts)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')


In [ ]:
# Calculate residuals (errors)
residuals = test_df['sales'][seq_length:] - test_forecasts

# Plot residuals
plt.figure(figsize=(10, 6))
plt.plot(test_df['date'][seq_length:], residuals, label='Residuals')
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Residuals over Time')
plt.xlabel('Date')
plt.ylabel('Residuals')
plt.legend()
plt.show()


In [ ]:
# Save the model
model.save_pretrained('gpt2_sales_forecasting')
tokenizer.save_pretrained('gpt2_sales_forecasting')

# Later, load the model
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained('gpt2_sales_forecasting')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2_sales_forecasting')


In [ ]:
def predict_future_sales(model, last_sequence, n_future_steps):
    model.eval()
    predictions = []
    input_seq = last_sequence

    for _ in range(n_future_steps):
        input_seq = input_seq.to(device)
        outputs = model.generate(input_seq)
        next_value = float(tokenizer.decode(outputs[0], skip_special_tokens=True).split()[-1])
        predictions.append(next_value)

        # Append the predicted value to the sequence for the next prediction
        input_seq = torch.cat([input_seq, torch.tensor([[next_value]]).to(device)], dim=1)

    return predictions

# Predict sales for the next 10 days
last_sequence = tokenized_train_sequences[-1]  # Use the last sequence from your training set
future_sales = predict_future_sales(model, last_sequence, n_future_steps=10)

# Inverse scale the future sales
future_sales = scaler.inverse_transform(np.array(future_sales).reshape(-1, 1))
print(future_sales)


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/forecast', methods=['POST'])
def forecast():
    data = request.get_json()
    # Extract input sequences from the data
    input_sequence = data['sequence']

    # Tokenize and predict sales using the model
    input_seq = torch.tensor([input_sequence]).to(device)
    output = model.generate(input_seq)

    prediction = float(tokenizer.decode(output[0], skip_special_tokens=True).split()[-1])
    return jsonify({'predicted_sales': prediction})

if __name__ == '__main__':
    app.run(debug=True)
